Importing Required Libraries 

In [8]:
import cv2
import mediapipe as mp
import numpy as np

Initialize MediaPipe Hand Tracking 

In [9]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

Initialize the drawing canvas (white background)

In [10]:
canvas_width, canvas_height = 800, 600  # Set canvas size
canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255  # White canvas
drawing = False  # Flag to track whether drawing is happening
previous_position = None  # To store previous finger position
drawn_lines = []  # Store the drawn lines for deletion

Function to draw on the canvas using the index finger

In [11]:
def draw_on_canvas(frame, landmarks, canvas):
    global previous_position, drawing, drawn_lines

    # Get the position of the index finger tip (landmark 8)
    index_finger_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]

    # Convert normalized coordinates to pixel values
    h, w, _ = frame.shape
    index_x = int(index_finger_tip.x * w)
    index_y = int(index_finger_tip.y * h)

    # Optional: Smoothing for more stable drawing
    if previous_position is not None:
        index_x = int((index_x + previous_position[0]) / 2)
        index_y = int((index_y + previous_position[1]) / 2)

    # If the finger is pointing, start drawing
    if previous_position is not None and drawing:
        # Draw a line from the previous finger position to the current one
        cv2.line(canvas, previous_position, (index_x, index_y), (0, 0, 255), 5)
        # Save the drawn line (to be able to delete later)
        drawn_lines.append((previous_position, (index_x, index_y)))

    # Update previous position to current position
    previous_position = (index_x, index_y)


Function to detect fist (for stopping drawing)

In [12]:
def is_fist(landmarks):
    # Check if the tips of all fingers (except thumb) are curled toward their base
    fingers_curled = 0
    fingers = [mp_hands.HandLandmark.INDEX_FINGER_TIP,
               mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
               mp_hands.HandLandmark.RING_FINGER_TIP,
               mp_hands.HandLandmark.PINKY_TIP]

    bases = [mp_hands.HandLandmark.INDEX_FINGER_DIP,
             mp_hands.HandLandmark.MIDDLE_FINGER_DIP,
             mp_hands.HandLandmark.RING_FINGER_DIP,
             mp_hands.HandLandmark.PINKY_DIP]

    for finger, base in zip(fingers, bases):
        if landmarks[finger].y > landmarks[base].y:  # Tip is below or closer to the base joint
            fingers_curled += 1

    # If all 4 fingers are curled, it's a fist
    return fingers_curled == 4

Function to detect two fingers raised (for deleting)

In [13]:
def is_two_fingers_raised(landmarks):
    # Peace sign gesture (two fingers raised)
    index_finger = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_finger = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    
    # Check if both fingers are pointing up
    if index_finger.y < landmarks[mp_hands.HandLandmark.INDEX_FINGER_DIP].y and \
       middle_finger.y < landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y:
        return True
    return False

Real-time webcam feed

In [14]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error accessing the camera.")
        break

    # Flip the frame to mirror the video feed
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB for MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame with MediaPipe Hands
    results = hands.process(rgb_frame)

    # Resize canvas to match the frame size
    canvas_resized = cv2.resize(canvas, (frame.shape[1], frame.shape[0]))

    # If hand landmarks are found, process them
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw the hand landmarks on the frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # If a fist is detected, stop drawing
            if is_fist(hand_landmarks.landmark):
                drawing = False
                previous_position = None
                cv2.putText(frame, 'Drawing Stopped (Fist)', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            elif is_two_fingers_raised(hand_landmarks.landmark):
                if drawn_lines:
                    drawn_lines.pop()  # Remove the last drawn line
                    canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
                    for line in drawn_lines:
                        cv2.line(canvas, line[0], line[1], (0, 0, 255), 5)
                cv2.putText(frame, 'Deleting Last Line (Two Fingers)', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                draw_on_canvas(frame, hand_landmarks.landmark, canvas)
                drawing = True
                cv2.putText(frame, 'Drawing...', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the frame with the drawing overlay
    cv2.imshow('Gesture Drawing', cv2.addWeighted(frame, 0.5, canvas_resized, 0.5, 0))

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()